## Vistual inertia scheduling

In [1]:
import andes
import os

from statistics import fmean
from andes.interop.pandapower import to_pandapower
from andes.interop.pandapower import make_GSF, build_group_table
import gurobipy as gb
import pandas as pd
import numpy as np
import logging

logger = logging.getLogger(__name__)

# from opf import dcopf
from visopf import vis1

## Main - IEEE 39


### load case from andes excel

In [2]:
# get andes case from excel
dir_path = os.path.abspath('..')
case_path = '/VIS_opf/ieee39_vis.xlsx'
case = dir_path + case_path
ssa = andes.load(case, no_output=True)

### load norm parameter

In [3]:
# prepare nn data for visopf
data_path = dir_path + '/VIS_opf/NN_train'

fnorm = pd.read_csv(data_path + '/fnorm.csv')
pnorm = pd.read_csv(data_path + '/pnorm.csv')
norm = {'fnorm': fnorm, 'pnorm': pnorm }
norm


{'fnorm':            M          D         Fg          Rg    fnadir
 0  19.895678  12.197409  80.138509  126.078088 -0.012637
 1   5.744600   4.344583  39.902459   43.025549  0.005493,
 'pnorm':            M          D         Fg          Rg      Mvsg      Dvsg     Ppeak
 0  19.895678  12.197409  80.138509  126.078088  0.981245  2.533602  0.067277
 1   5.744600   4.344583  39.902459   43.025549  0.587552  0.867664  0.036406}

In [4]:
# example to call norm data
norm['fnorm']['M'].iloc[0] # mean
norm['fnorm']['M'].iloc[1] # std 

5.74459957980676

### load nn parameter

In [5]:
fw1 = pd.read_csv(data_path + '/fw1.csv', header=None)
fw2 = pd.read_csv(data_path + '/fw2.csv', header=None)

fb1 = pd.read_csv(data_path + '/fb1.csv', header=None)
fb2 = pd.read_csv(data_path + '/fb2.csv', header=None)

In [6]:
pw1 = pd.read_csv(data_path + '/pw1.csv', header=None)
pw2 = pd.read_csv(data_path + '/pw2.csv', header=None)

pb1 = pd.read_csv(data_path + '/pb1.csv', header=None)
pb2 = pd.read_csv(data_path + '/pb2.csv', header=None)

In [7]:
nn = {
        'fw1': fw1,
        'fw2': fw2,       
        'fb1': fb1,
        'fb2': fb2,
        'pw1': pw1,
        'pw2': pw2,
        'pb1': pb1,
        'pb2': pb2,
    }

### test opf model

In [8]:
ss = vis1(norm=norm, nn=nn, dpe=0.05)
ss.norm

Restricted license - for non-production use only - expires 2023-10-25


{'fnorm':            M          D         Fg          Rg    fnadir
 0  19.895678  12.197409  80.138509  126.078088 -0.012637
 1   5.744600   4.344583  39.902459   43.025549  0.005493,
 'pnorm':            M          D         Fg          Rg      Mvsg      Dvsg     Ppeak
 0  19.895678  12.197409  80.138509  126.078088  0.981245  2.533602  0.067277
 1   5.744600   4.344583  39.902459   43.025549  0.587552  0.867664  0.036406}

In [9]:
ss.norm['fnorm']['M'].iloc[0]

19.895678

In [10]:
ss.nn['fw1']

,0,1,2,3
0,0.053316,-0.018950,0.103133,0.005192
1,0.090899,-0.021882,-0.052110,-0.066491
2,-0.054552,0.081501,0.133793,0.073621
3,0.013614,0.007082,0.004717,-0.008138
4,0.017724,-0.007474,0.009466,-0.007022
...,...,...,...,...
59,0.009615,-0.005689,-0.002588,-0.018844
60,0.064262,-0.043027,0.112759,-0.003101
61,-0.140001,-0.125340,-0.942938,-0.133099
62,0.006305,-0.006394,-0.003145,0.004949


In [11]:
ss.nn['fw1'][1].iloc[1]
# ss.nn['fb1']

-0.021882491

In [12]:
vsg_ieee14 = ['PV_6', 'PV_7']
vsg_ieee39 = ['PV_1', 'PV_6', 'PV_8', 'PV_9']

ss.from_andes(ssa, vsg_ieee39)

In [13]:
ss.gen

,idx,u,name,Sn,Vn,bus,p0,pmax,pmin,v0,...,ramp30,type,p_pre,band,K,M,D,R,Mvsg,Dvsg
0,PV_1,1.0,PV_1,10.400,34.5,30,4.360864,15.0,2.0,1.035534,...,600,2,0,13.0,1,0.00000,0.0,0.000000,104.000,52.000
1,PV_2,1.0,PV_2,8.360,34.5,31,6.460000,9.0,1.5,1.013246,...,600,1,0,7.5,1,67.38160,0.0,0.005981,0.000,0.000
2,PV_3,1.0,PV_3,8.437,21.0,32,7.250000,8.0,1.0,1.020528,...,600,1,0,7.0,1,60.40892,0.0,0.005926,0.000,0.000
3,PV_4,1.0,PV_4,11.748,21.0,33,6.520000,7.0,1.0,1.013430,...,600,1,0,6.0,1,102.44256,0.0,0.004256,0.000,0.000
4,PV_5,1.0,PV_5,10.802,15.5,34,5.200000,7.0,1.0,1.019109,...,600,1,0,6.0,1,56.17040,0.0,0.004629,0.000,0.000
5,PV_6,1.0,PV_6,10.857,15.5,35,6.900000,8.0,1.0,1.060000,...,600,2,0,7.0,1,0.00000,0.0,0.000000,65.142,54.285
6,PV_7,1.0,PV_7,10.252,12.5,36,5.900000,7.0,1.0,1.060000,...,600,1,0,6.0,1,84.88656,0.0,0.004877,0.000,0.000
7,PV_8,1.0,PV_8,9.702,12.5,37,3.300000,7.0,1.0,1.013996,...,600,2,0,6.0,1,0.00000,0.0,0.000000,77.616,29.106
8,PV_9,1.0,PV_9,16.841,34.5,38,7.800000,10.0,1.0,1.052803,...,600,2,0,9.0,1,0.00000,0.0,0.000000,101.046,84.205
9,Slack_10,1.0,Slack_10,11.990,345.0,39,5.741700,15.0,3.0,1.030000,...,600,1,0,12.0,1,119.90000,0.0,0.004170,0.000,0.000


In [14]:
# ss.gendict

In [15]:
ss.norm

{'fnorm':            M          D         Fg          Rg    fnadir
 0  19.895678  12.197409  80.138509  126.078088 -0.012637
 1   5.744600   4.344583  39.902459   43.025549  0.005493,
 'pnorm':            M          D         Fg          Rg      Mvsg      Dvsg     Ppeak
 0  19.895678  12.197409  80.138509  126.078088  0.981245  2.533602  0.067277
 1   5.744600   4.344583  39.902459   43.025549  0.587552  0.867664  0.036406}

In [16]:
a_test = []
for i in range(64):
    a_test.append('a'+str(i))
# a_test

In [17]:
ss.gendict.keys()

dict_keys(['PV_1', 'PV_2', 'PV_3', 'PV_4', 'PV_5', 'PV_6', 'PV_7', 'PV_8', 'PV_9', 'Slack_10'])

In [18]:
ss.nn['fw2'][4].iloc[0]

0.009975906

In [19]:
ss.nn['fb1']

,0
0,0.288764
1,0.193274
2,0.329569
3,-0.045777
4,-0.388495
...,...
59,-0.407754
60,0.345564
61,-1.168843
62,-0.120155


In [20]:
ss.nn['fb2'][0].iloc[0]

-0.09059838

In [21]:
ss.build()

# ss.mdl.optimize()

Successfully build var.
Successfully build obj.
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Successfully build cons.


In [23]:
ss.GENII_test

dict_keys(['PV_1', 'PV_6', 'PV_8', 'PV_9'])

In [24]:
ss.get_res()

Successfully build var.
Successfully build obj.
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Successfully build cons.
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[arm])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 264 rows, 166 columns and 2790 nonzeros
Model fingerprint: 0x28c4263e
Variable types: 102 continuous, 64 integer (64 binary)
Coefficient statistics:
  Matrix range     [2e-07, 1e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 2e+01]
  RHS range        [9e-03, 1e+02]
Presolve removed 211 rows and 134 columns
Presolve time: 0.00s
Presolved: 53 rows, 32 columns, 399 nonzeros
Variable types: 22 continuous, 10 integer (10 binary)
Found heuristic solution: objective 58.5640000

Root relaxation: cutoff, 0 iterations, 0.00 seconds (0.00 work units)

Explored 1 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 8 (of 8 available processors)

Sol

Successfully solve vis1.


,gen,pg,pru,prd
0,PV_1,11.620558,0.0,0.0
1,PV_2,2.865036,0.0,0.0
2,PV_3,7.312019,0.0,0.0
3,PV_4,7.000000,0.0,0.0
4,PV_5,3.800000,0.0,0.0
5,PV_6,8.000000,0.0,0.0
6,PV_7,3.061945,0.0,0.0
7,PV_8,7.000000,0.0,0.0
8,PV_9,4.904442,0.0,0.0
9,Slack_10,3.000000,0.0,0.0


In [25]:
ss.load.p0.sum()

ss.rocof

ss.Msys # check whether dpe is reasonable

<gurobi.LinExpr: 47.455284378136746 + 0.09507354487197069 Mvsg[PV_1] + 0.09925129583413325 Mvsg[PV_6] + 0.08869264734114035 Mvsg[PV_8] + 0.15395515088354406 Mvsg[PV_9]>

In [26]:
ss.costdict

{'PV_1': {'c2': 0, 'c1': 1, 'c0': 0, 'cr': 0, 'cru': 0, 'crd': 0},
 'PV_2': {'c2': 0, 'c1': 1, 'c0': 0, 'cr': 0, 'cru': 0, 'crd': 0},
 'PV_3': {'c2': 0, 'c1': 1, 'c0': 0, 'cr': 0, 'cru': 0, 'crd': 0},
 'PV_4': {'c2': 0, 'c1': 1, 'c0': 0, 'cr': 0, 'cru': 0, 'crd': 0},
 'PV_5': {'c2': 0, 'c1': 1, 'c0': 0, 'cr': 0, 'cru': 0, 'crd': 0},
 'PV_6': {'c2': 0, 'c1': 1, 'c0': 0, 'cr': 0, 'cru': 0, 'crd': 0},
 'PV_7': {'c2': 0, 'c1': 1, 'c0': 0, 'cr': 0, 'cru': 0, 'crd': 0},
 'PV_8': {'c2': 0, 'c1': 1, 'c0': 0, 'cr': 0, 'cru': 0, 'crd': 0},
 'PV_9': {'c2': 0, 'c1': 1, 'c0': 0, 'cr': 0, 'cru': 0, 'crd': 0},
 'Slack_10': {'c2': 0, 'c1': 1, 'c0': 0, 'cr': 0, 'cru': 0, 'crd': 0}}

In [27]:
ss.zf


{'zf0': <gurobi.Var zf[zf0] (value 0.645879963683487)>,
 'zf1': <gurobi.Var zf[zf1] (value 0.5723249740820853)>,
 'zf2': <gurobi.Var zf[zf2] (value 0.0)>,
 'zf3': <gurobi.Var zf[zf3] (value 0.0)>,
 'zf4': <gurobi.Var zf[zf4] (value 0.0)>,
 'zf5': <gurobi.Var zf[zf5] (value 1.0404143220275082)>,
 'zf6': <gurobi.Var zf[zf6] (value 0.0)>,
 'zf7': <gurobi.Var zf[zf7] (value 0.0)>,
 'zf8': <gurobi.Var zf[zf8] (value 0.0)>,
 'zf9': <gurobi.Var zf[zf9] (value 0.0)>,
 'zf10': <gurobi.Var zf[zf10] (value 0.0)>,
 'zf11': <gurobi.Var zf[zf11] (value 0.0)>,
 'zf12': <gurobi.Var zf[zf12] (value 0.0)>,
 'zf13': <gurobi.Var zf[zf13] (value 0.0)>,
 'zf14': <gurobi.Var zf[zf14] (value 0.0)>,
 'zf15': <gurobi.Var zf[zf15] (value 0.0)>,
 'zf16': <gurobi.Var zf[zf16] (value 0.0)>,
 'zf17': <gurobi.Var zf[zf17] (value 0.22398106511631102)>,
 'zf18': <gurobi.Var zf[zf18] (value 0.0)>,
 'zf19': <gurobi.Var zf[zf19] (value 0.0)>,
 'zf20': <gurobi.Var zf[zf20] (value 0.7253476851963944)>,
 'zf21': <gurobi.Var 

In [28]:
ss.mdl.display()

Minimize
<gurobi.LinExpr: pg[PV_1] + pg[PV_2] + pg[PV_3] + pg[PV_4] + pg[PV_5] + pg[PV_6]
+ pg[PV_7] + pg[PV_8] + pg[PV_9] + pg[Slack_10]>
Subject To
PowerBalance: <gurobi.LinExpr: pg[PV_1] + pg[PV_2] + pg[PV_3] + pg[PV_4] + pg[PV_5] +
 pg[PV_6] + pg[PV_7] + pg[PV_8] + pg[PV_9] + pg[Slack_10]> = 58.564
Line_1_U: <gurobi.LinExpr: -0.5607822357546095 pg[PV_1] + -0.6089602208400834 pg[PV_2]
+ -0.398309791005728 pg[PV_3] + -0.4219560834463752 pg[PV_4] + -0.5066725368514862
pg[PV_5] + -0.5066725368514862 pg[PV_6] + -0.5066725368514856 pg[PV_7] +
-0.5066725368514856 pg[PV_8] + -0.5988298865678321 pg[PV_9] + -0.5607822357546095
 pg[Slack_10]> <= -20.9585
Line_1_D: <gurobi.LinExpr: -0.5607822357546095 pg[PV_1] + -0.6089602208400834 pg[PV_2]
+ -0.398309791005728 pg[PV_3] + -0.4219560834463752 pg[PV_4] + -0.5066725368514862
pg[PV_5] + -0.5066725368514862 pg[PV_6] + -0.5066725368514856 pg[PV_7] +
-0.5066725368514856 pg[PV_8] + -0.5988298865678321 pg[PV_9] + -0.5607822357546095
 pg[Slack_10]> >= -

In [29]:
ss.Mvsg['PV_1'].X

4.0

In [31]:
ss.Mvsg['PV_8'].X

0.4730679682566654

In [32]:
ss.Mvsg['PV_6'].X

4.0

In [33]:
ss.Mvsg['PV_9'].X

4.0